# Python Pattern Matching

First, let's use Jupyter Lab, a more up to date version of Jupyter Notebooks, for this demo.

We open a terminal shell. Make a new directory for "Pattern-Matching". Pattern Matching requires Python v 3.10 or better, so let's make sure `uv` is using a modern enough Python for us. Remember with `uv` we can just tell it what version of Python we need and `uv` will take care of installing whatever Python or packages we need.
```
$ pwd # What folder are we in?
/home/peter/projects/hacker-dojo-python-group/PatternMatching

$ ls -a # Do we have a .venv and a pyproject.toml?

$ # No there isn't. uv will look for a .venv in a parent directory.
$ uv run python -c "import os,sys; print(os.environ.get('VIRTUAL_ENV') or sys.prefix)"` # Which .venv is uv using? 
/home/peter/projects/hacker-dojo-python-group/.venv

$ uv run which python; uv run python --version  # show what Python uv is using
/home/peter/projects/hacker-dojo-python-group/.venv/bin/python
Python 3.13.2

$ # Python 3.13 is good enough for us. We are going to install jupyterlab. We are ok installing it in the shared hacker-dojo projects .venv.
$ uv run python -m jupyterlab # Start Juptyer Lab notebook to open THIS notebook we are looking at.

$ # If we wanted to separate Pattern-Matching from the rest of the hacker-dojo demos, we could create our own .venv here in this directory: uv init --bare; uv venv; uv add jupyterlab
```

## Python Pattern Matching

Pattern Matching is a relatively new (Python 3.10) feature that gives us "switch/case" and "structural pattern matching"

In [4]:
import argparse
import shlex

def main( args_string:str=None):
    parser = argparse.ArgumentParser()
    parser.add_argument('command', choices=['push', 'pull', 'commit'])
    if args_string:
        args = parser.parse_args(shlex.split(args_string))
    else:
        args = parser.parse_args() # parse command line ARGV

    match args.command:
        case 'push':
            print('pushing')
        case 'pull':
            print('pulling')
        case _:
            parser.error(f'{args.command!r} not yet implemented')

main("push")
   

pushing


Note: Python still allows the name "match" and "case" for variable names. They are only special keywords in this syntax.

## The case patterns don’t just have to be literals. The patterns can also:  

- Use variable names that are set if a case matches  
- Match sequences using list or tuple syntax (like Python’s existing iterable unpacking feature)  
- Match mappings using dict syntax  
- Use * to match the rest of a list  
- Use ** to match other keys in a dict  
- Match objects and their attributes using class syntax  
- Include “or” patterns with |  
- Capture sub-patterns with as  
- Include an if “guard” clause  

In [6]:
# Let's make a function that uses all of those features:

def match_demo( expr):
    match expr:
        case (0, x):              # seq of 2 elems with first 0
            print(f'(0, {x}); found {x}')    # (new variable x set to second elem)
        case ['a', x, 'c']:       # seq of 3 elems: 'a', anything, 'c'
            print(f"'a', {x!r}, 'c'; found{x!r}")
        case {'foo': bar}:        # dict with key 'foo' (may have others)
            print(f"{{'foo': {bar}}} found {bar}")
        case [1, 2, *rest]:       # seq of: 1, 2, ... other elements
            print(f'[1, 2, star{rest}] found {rest}')
        case {'x': x, **kw}:      # dict with key 'x' (others go to kw)
            print(f"{{'x': {x}, double-star{kw}}} found {x} and {kw}")
        case Car(key=key, name='Tesla'):  # Car with name 'Tesla' (any key)
            print(f"Car({key!r}, 'TESLA!')")
        case Car(key, name):      # similar to above, but use __match_args__
            print(f"Car({key!r}, {name!r})")
        case 1 | 'one' | 'I':     # int 1 or str 'one' or 'I'
            print('one')
        case ['a'|'b' as ab, c]:  # seq of 2 elems with first 'a' or 'b'
            print(f'{ab!r}, {c!r} found {c!r}')
        case (x, y) if x == y:    # seq of 2 elems with first equal to second
            print(f'({x}, {y}) with x==y')
        case _:
            print('no match')

# and a sample Class to show matching object attributes

class Car:
    __match_args__ = ('key', 'name')
    def __init__(self, key, name):
        self.key = key
        self.name = name

In [7]:
match_demo( (0, 'treasure') )

(0, treasure); found treasure


In [9]:
match_demo( ('a', 'treasure', 'c') )
match_demo( ('a', 'treasure', 'c', 'd') )

'a', 'treasure', 'c'; found'treasure'
no match


In [11]:
match_demo( {'foo':'treasure', 'other_key': 'other_value'} )
match_demo( {'xxx':'treasure', 'other_key': 'other_value'} )

{'foo': treasure} found treasure
no match


In [12]:
match_demo( [1, 2, 'here', 'are', 'some', 'more', 'values'])

[1, 2, star['here', 'are', 'some', 'more', 'values']] found ['here', 'are', 'some', 'more', 'values']


In [13]:
match_demo( {'x': 'x-value', 'y': 'y-value', 'z': 'z-value'} )

{'x': x-value, double-star{'y': 'y-value', 'z': 'z-value'}} found x-value and {'y': 'y-value', 'z': 'z-value'}


In [14]:
my_car = Car(key='ABCXYZ', name='Tesla')
match_demo( my_car)

Car('ABCXYZ', 'TESLA!')


In [15]:
my_other_car = Car(key="123456", name="Volkswagen")
match_demo( my_other_car)

Car('123456', 'Volkswagen')


In [29]:
match_demo( 1)
match_demo( 'one')
match_demo( 'I')
match_demo( 1.0)
match_demo( 'won')
match_demo( 'i')
match_demo( True)

one
one
one
one
no match
no match
one


In [17]:
match_demo( ('a', 'zebra'))
match_demo( ('b', 'zebra'))

'a', 'zebra' found 'zebra'
'b', 'zebra' found 'zebra'


In [28]:

match_demo( [99, 99])
match_demo( ("same", "same"))
match_demo( ( [1,2,3], [1,2,3]))
match_demo( [ ({'a':2,'b':3}, set("abcd"), 1.2), ({'b':3,'a':2}, set("dacb"), 1.2)])

no match
(99, 99) with x==y
(same, same) with x==y
([1, 2, 3], [1, 2, 3]) with x==y
(({'a': 2, 'b': 3}, {'b', 'c', 'a', 'd'}, 1.2), ({'b': 3, 'a': 2}, {'c', 'b', 'a', 'd'}, 1.2)) with x==y




Using this as a nice syntax match/case to replace a bunch of if/elif/else is nice, but the real power comes from using it to handle more complex "dispatching" of multiple actions
Here is an example of a text advanture game:

In [64]:
def game_loop():
    command = input("What are you doing next? ")
    match command.split():
        case ["quit"]:
            print("Goodbye!")
            quit_game()
        case ["look"]:
            current_room.describe()
        case ["get", obj]:
            character.get(obj, current_room)
        case ["drop", *objects]:
            for obj in objects:
                character.drop(obj, current_room)
        case ["go", direction] if direction in current_room.exits:
            current_room = current_room.neighbor(direction)
        case ["go", _]:
            print("Sorry, you can't go that way")
        case _:
            print(f"Sorry, I couldn't understand {command!r}")

Or another example from a video-game, where you have events of different classes such as KeyPress and mouse Click objects. These pattern matches replace a lot of if conditions.

In [ ]:
match event.get():
    case Click((x, y), button=Button.LEFT):  # This is a left click
        handle_click_at(x, y)
    case Click():
        pass  # ignore other clicks
    case KeyPress(key_name="Q") | Quit():
        game.quit()
    case KeyPress(key_name="up arrow"):
        game.go_north()
    ...
    case KeyPress():
        pass # Ignore other keystrokes
    case other_event:
        raise ValueError(f"Unrecognized event: {other_event}")

Cool example using it to pattern match deep inside JSON:

In [67]:
import json

def log(event):
    match json.loads(event):
        case {"keyboard": {"key": {"code": code}}}:
            print(f"Key pressed: {code}")
        case {"mouse": {"cursor": {"screen": [x, y]}}}:
            print(f"Mouse cursor: {x=}, {y=}")
        case _:
            print("Unknown event type")

log( json.dumps( {'keyboard':{'key':{'code': "Escape"}}}))
log( json.dumps( {'mouse':{'cursor':{'screen':[1280,640]}}}))

Key pressed: Escape
Mouse cursor: x=1280, y=640


### Caution!  
Variables names in case expressions are used to bind found values to the variable. Anywhere else in a Python program a variable name will be substituted with its value.
```
case: ( 'pixel', x)
```
Will NOT match the value of `x`, it will set `x` to the second value found after the first value `pixel`. This is different from using a variable name anywhere else.  
  
Q: So how do you match a constant in a variable, like if you had the constants RED=(255,0,0) BLUE=(0,0,255) GREEN=(0,255,0)?
```
case ('pixel', RED)
```
Will NOT work, it would set the variable RED to the found value after 'pixel'.
```
case ('pixel', Color.RED)
```
This will work. If the variable is a fully qualified name with a DOT in it, it will use the value for matching.    
Think of `case` patterns more like function declarations than expressions. You are listing literals and names, not giving expressions.

## Resources:
[PEP 634](https://www.python.org/dev/peps/pep-0634/): the specification  
[PEP 635](https://www.python.org/dev/peps/pep-0635/): motivation and rationale  
[PEP 636](https://www.python.org/dev/peps/pep-0636/): a tutorial for the feature  